### Agents and Tools

#### Agents: Agents enables LLMs to work with tools that perform various tasks

To use agents, we require three things:
 - A base LLM
 - A tool that we will be interacting with,
 - An agent to control the interaction.

#### In the following example, we will use the Math tool to perform some simple math operations

In [3]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()

True

In [4]:
import os

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0.5, 
    google_api_key=os.getenv("GOOGLE_API_KEY"))

In [2]:
#Chain that interprets a prompt and executes python code to do math
from langchain.chains import LLMMathChain 
from langchain.agents import Tool

In [8]:
# initialize the math tool
llm_math = LLMMathChain(llm=llm)

math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer math questions.'
)

tools = [math_tool]

/Applications/anaconda3/envs/rag_env/lib/python3.9/site-packages/langchain/chains/llm_math/base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [17]:
print(tools[0].name,"---", tools[0].description)

Calculator --- Useful for when you need to answer math questions.


#### Now create an Agent
##### Zero-shot ReAct
The Zero-shot Reacr Agent is a language generation model that can create realistic contexts even without being traineed on specific data. It can be used for various tasks such as generating creative text formats, language transalation, and generating different types of creative contents.

In [8]:
from langchain.agents import initialize_agent

In [15]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

In [16]:
zero_shot_agent("What is root over 25")

/var/folders/y_/0dh_vnhx4h9bd_bn8cgqtnd00000gn/T/ipykernel_12900/2229265460.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  zero_shot_agent("What is root over 25")




> Entering new AgentExecutor chain...
The question is asking for the square root of 25.
Action: Calculator
Action Input: sqrt(25)
Observation: Answer: 5.0
Thought:I now know the final answer
Final Answer: 5.0

> Finished chain.


{'input': 'What is root over 25', 'output': '5.0'}

In [ ]:
problem = """
The ratio of the number of boys to the number of girls was 3: 5. 
Each boy was given 2 stickers and each girl was given 3. 
The girls received 81 more stickers than the boys. 
How many stickers did the boys receive in total?
"""

In [19]:
zero_shot_agent(problem)



> Entering new AgentExecutor chain...
Let $b$ be the number of boys and $g$ be the number of girls.
We are given that the ratio of boys to girls is 3:5, so $\frac{b}{g} = \frac{3}{5}$.
We can write $b = 3x$ and $g = 5x$ for some value of $x$.
Each boy received 2 stickers, so the boys received $2b$ stickers in total.
Each girl received 3 stickers, so the girls received $3g$ stickers in total.
The girls received 81 more stickers than the boys, so $3g = 2b + 81$.
Substituting $b = 3x$ and $g = 5x$ into the equation $3g = 2b + 81$, we get:
$3(5x) = 2(3x) + 81$
$15x = 6x + 81$
$15x - 6x = 81$
$9x = 81$
$x = \frac{81}{9}$
$x = 9$
Now we can find the number of boys and girls:
$b = 3x = 3(9) = 27$
$g = 5x = 5(9) = 45$
The boys received $2b$ stickers, so they received $2(27) = 54$ stickers.
The girls received $3g$ stickers, so they received $3(45) = 135$ stickers.
The difference in the number of stickers is $135 - 54 = 81$, which matches the given information.

The number of stickers the boys

{'input': 'The ratio of the number of boys to the number of girls was 3: 5. Each boy was given 2 stickers and each girl was given 3. The girls received 81 more stickers than the boys. How many stickers did the boys receive in total?',
 'output': 'The final answer is $\\boxed{54}$'}

#### Using multiple Tools
Using Duck Duck Go Search API as a Tool combined with the Math Tool We will then give the AGent a problem that initiates a search first followed by a Calculation.

In [5]:
# Define Tools

from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func = search.run,
    name = "Search",
    description = "Useful for when you need to seach the internet."
)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
math_tool = Tool.from_function(
    func = llm_math_chain.run,
    name = "Calculator",
    description = "Useful for when you need to answer math questions."
)

/var/folders/y_/0dh_vnhx4h9bd_bn8cgqtnd00000gn/T/ipykernel_81269/1216458497.py:12: LangChainDeprecationWarning: This class is deprecated and will be removed in langchain 1.0. See API reference for replacement: https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html
  llm_math_chain = LLMMathChain(llm=llm, verbose=True)
/Applications/anaconda3/envs/rag_env/lib/python3.9/site-packages/langchain/chains/llm_math/base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [10]:
# Define the Agent
tools = [search_tool, math_tool]
agent = initialize_agent(
    tools, 
    llm, 
    agent ="zero-shot-react-description", 
    verbose=True
)

In [26]:
# Run the Agent
agent.run("""Get Gold Prices taken from google finance and display in SGD.""")



> Entering new AgentExecutor chain...
I need to get the gold prices from Google Finance and convert them to SGD. I will start by searching for the current gold price in USD and then convert it to SGD.
Action: Search
Action Input: "current gold price USD google finance"
Observation: GOLDPRICE.ORG - The number 1 web site for United States spot gold price charts in ounces, grams and kilos. The spot price of gold is the most common standard used to gauge the going rate for a troy ounce of gold. The price is driven by speculation in the markets, currency values, current events, and many other factors. Gold spot price is used as the basis for most bullion dealers to determine the exact price to charge for a specific coin or bar. These prices are calculated in troy ounces and change ... Gold Price in USA Today: per oz 3,371.04 United States dollars. Gold prices for the USA are denominated in American dollars (USD). Live Price of Gold provides daily updates on gold prices across various stat

'The current gold price is approximately 4369.54 SGD per ounce.'

In [28]:
# Run the Agent
agent.run("""Get stock price of Tesla taken from google finance determin if its a good time to buy in.""")



> Entering new AgentExecutor chain...
I need to get the stock price of Tesla from Google Finance and then determine if it's a good time to buy. I will start by searching for the stock price. After getting the stock price, I will need to analyze the information to determine if it's a good time to buy. I will need to look for recent news and analysis related to Tesla's stock.

Action: Search
Action Input: "Tesla stock price Google Finance"
Observation: Get a real-time Tesla, Inc. (TSLA) stock price with an overview of financials, statistics, forecasts, charts and more. Discover real-time Tesla, Inc. Common Stock (TSLA) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions. Stay ahead with Nasdaq. Track TESLA, INC. (TSLA) price, historical values, financial information, price forecast, and insights to empower your investing journey | MSN Money Tesla (NASDAQ: TSLA) is facing a rough start to 2025, struggling to sustain the momentum that f

"It is difficult to definitively say if it's a good time to buy Tesla stock. Analyst ratings are mixed, and the stock is trading at a high valuation. It depends on your investment strategy and risk tolerance."

### Create a custom tool

In [30]:
from langchain.tools import BaseTool
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools import tool

@tool("lower_case", return_direct=True)
def StructuredResponseTool(question:str):
    """
    use this tool to send a prompt and get a JSON returned
    with three fields - Topic, Question_Details and Detailed_response
    """
    json_prompt = PromptTemplate.from_template(
        """
        Return a JSON object with an 'answer' key that answers the following question:{question}.
        The JSON object will hve threee fields - Topic, Question_Details and Detailed_response."""
    )

    llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0.5, 
    google_api_key=os.getenv("GOOGLE_API_KEY"))

    json_parser = JsonOutputParser()
    json_chain = json_prompt | model | json_parser
    x = json_chain.invoke({"question": question})
    return x


In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import initialize_agent

In [32]:
tools= [StructuredResponseTool]

# Initialize the agent with the tools
zero_shot_agent = initialize_agent(
    agent ="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10)

In [33]:
zero_shot_agent.run(input="Which is the top us stock in the world now and how, the response should have three field?")



> Entering new AgentExecutor chain...
I need to find the top US stock in the world and explain how it achieved that position. I should use the `lower_case` tool to get the information. The response should have three fields.
Action: lower_case
Action Input: Which is the top US stock in the world now and how, the response should have three fields Topic, Question_Details and Detailed_response?
Observation: {'Topic': 'US Stock Market', 'Question_Details': 'Identify the top US stock in the world currently and explain the criteria used to determine its ranking.', 'Detailed_response': {'answer': 'Determining the "top" US stock depends on the criteria used for evaluation. Here are a few common metrics and the corresponding top contenders, along with explanations:\n\n*   **By Market Capitalization:**  As of October 27, 2024 (This date is important as the information changes frequently), **Microsoft (MSFT)** and **Apple (AAPL)** are consistently battling for the top spot in terms of market cap

{'Topic': 'US Stock Market',
 'Question_Details': 'Identify the top US stock in the world currently and explain the criteria used to determine its ranking.',
 'Detailed_response': {'answer': 'Determining the "top" US stock depends on the criteria used for evaluation. Here are a few common metrics and the corresponding top contenders, along with explanations:\n\n*   **By Market Capitalization:**  As of October 27, 2024 (This date is important as the information changes frequently), **Microsoft (MSFT)** and **Apple (AAPL)** are consistently battling for the top spot in terms of market capitalization. Market capitalization is calculated by multiplying the company\'s share price by the number of outstanding shares.  Higher market capitalization generally indicates a larger and more valuable company.\n\n*   **By Stock Performance (Year-to-Date or Recent Period):** The "top" stock based on performance changes constantly.  For example, a smaller growth stock might have significantly higher pe

{'Topic': 'US Stock Market',
 'Question_Details': 'Identify the top US stock in the world currently and explain the criteria used to determine its ranking.',
 'Detailed_response': {'answer': 'Determining the "top" US stock depends on the criteria used for evaluation. Here are a few common metrics and the corresponding top contenders, along with explanations:\n\n*   
 
 **By Market Capitalization:**  As of October 27, 2024 (This date is important as the information changes frequently), 
 
 **Microsoft (MSFT)** and 
 
 **Apple (AAPL)** are consistently battling for the top spot in terms of market capitalization. Market capitalization is calculated by multiplying the company\'s share price by the number of outstanding shares.  Higher market capitalization generally indicates a larger and more valuable company.\n\n*   
 
 **By Stock Performance (Year-to-Date or Recent Period):** The "top" stock based on performance changes constantly.  For example, a smaller growth stock might have significantly higher percentage gains over a shorter period than a mega-cap like Apple or Microsoft. To determine the top performer based on stock performance, you\'d need to consult financial data providers (e.g., Google Finance, Yahoo Finance, Bloomberg, Refinitiv) and filter by percentage gain over a specific timeframe.\n\n*  
 
  **By Revenue or Profitability:**  Companies like **Amazon (AMZN)** or **Walmart (WMT)** often lead in terms of revenue.  Profitability can vary widely depending on the sector and economic conditions.  Again, financial data sources are needed to identify the top companies based on these metrics.\n\n*   
  
  **By Analyst Ratings and Future Growth Potential:** Many investors consider analyst ratings and projected growth when assessing a stock\'s potential. Companies in sectors like artificial intelligence, cloud computing, and renewable energy often receive high ratings due to their growth prospects. However, these ratings are subjective and based on analysts\' opinions.\n\n
  
  **In summary:** There is no single definitive "top" US stock.  
  
  **Microsoft and Apple are typically considered the leaders based on market capitalization**, but other metrics like stock performance, revenue, profitability, and growth potential can lead to different conclusions.  The best approach is to define your criteria for "top" and then research companies that meet those criteria using reliable financial data sources. Always consult a financial professional before making investment decisions.'}}